In [ ]:
# パッケージのインストール
%pip install -q langchain
%pip install -q langchain-openai
%pip install -q langchain-chroma
%pip install -q langchain-community
%pip install -q python-dotenv
%pip install -q pdfminer.six
%pip install -q chromadb
%pip install -q ipywidgets
%pip install -q Jinja2
%pip install -q pandas
%pip install -q ragas datasets
%pip install -q matplotlib


In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [6]:
chunk_size = 200
collection_name = f"mycollection_{chunk_size}_ef_100"
embedding_model = "text-embedding-3-large"

top_k = 10
batch_size = 500

query="高町なのはがユーノから譲り受けたものは何ですか？"

### 埋め込みモデルの準備

In [7]:
from langchain_openai import OpenAIEmbeddings

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### ドキュメントの読み込みとチャンキング

In [8]:
from langchain_community.document_loaders import PDFMinerLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ドキュメントの読み込み
loader = DirectoryLoader(
    "./data/",
    glob="**/*.pdf",  # PDFファイルを対象に
    show_progress=True,
    loader_cls=PDFMinerLoader  # PDFMinerLoaderを使用
)

In [9]:
# ドキュメントの分割
documents = []
for doc in loader.load():
    # すべての改行を削除（単語の途中でチャンキングされることを防ぐため）
    doc.page_content = doc.page_content.replace("\n", "")
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        separators=[
            "。",
            "、",
            " ",
            "」",
            "）",
            "｝",
            "〕",
            "》"
        ],
        keep_separator="end"
    ).split_documents([doc])
    documents.extend(chunks)

print(f"合計チャンク数: {len(documents)}")

100%|██████████| 19/19 [00:46<00:00,  2.46s/it]

合計チャンク数: 3378


In [10]:
for i, doc in enumerate(documents[:5]):
    print(f"\n=== Document {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")


=== Document 1 ===
Content: ハリー‧ポッター (架空の⼈物)出典: フリー百科事典『ウィキペディア（Wikipedia）』ハリー‧ポッターシリーズ > ハリー‧ポッターシリーズの登場⼈物⼀覧 > ハリー‧ポッター (架空の⼈物)ハリー‧ジェームズ‧ポッター（英: Harry JamesPotter）は、J‧K‧ローリングの⼩説『ハリー‧ポッター』シリーズおよび、その派⽣作品に登場する架空の⼈物であり、同シリーズの主⼈公。
文字数: 198
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 2 ===
Content: ホグワーツ魔法魔術学校グリフィンドール寮の男⼦⽣徒となる。孤児として⺟親の親類の伯⺟夫婦の家で不遇な暮らしをして育った。11歳を迎える年のある⽇突然、ホグワーツから⼊学許可証が届いたのをきっかけに、亡くなった両親が魔法使いであったこと、そして出⽣時に下された予⾔により、闇の魔法使いヴォルデモートを倒す宿命を⾃分が負っていると告げられる。
文字数: 169
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 3 ===
Content: マグル界では⼀介の少年に過ぎない⽣活を送っていたが、魔法界では本⼈が⼾惑うほど重要な⼈物として、あまねく⼈々から知られている。⼀⼈前の魔法使いになるべく、同級⽣のロン‧ウィーズリーやハーマイオニー‧グレンジャーらとともに、ホグワーツにて学⽣⽣活を送りつつ、宿敵のヴォルデモートなどの闇の魔法使いたちによる数々の陰謀に⽴ち向かう冒険の⽇々を通して、たくましく成⻑していく姿が物語で描かれている。
文字数: 196
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 4 ===
Content: ⼈物名前‧外⾒魔 法 界 で は 「 ⽣ き 残 っ た 男 の ⼦   (The  boy  wholived) 」と呼ばれる。髪の⽑は黒い癖⽑で、瞳は明るい緑⾊。⼩顔で細⾯で、近視のため丸眼鏡を着⽤。同年代に⽐べ⼩柄で痩せているが、第6巻『謎のプリンス』では前巻と⽐べて⾝⻑がかなり伸びたとされている。
文字数: 

### VectorStoreの準備

In [11]:
import chromadb
from langchain_chroma import Chroma


# Chromaクライアントを初期化
try:
    persistent_client = chromadb.PersistentClient(path="./chroma_db")
    # 既存のChromaコレクションを削除
    try:
        persistent_client.delete_collection(collection_name)
        print(f"既存のコレクション '{collection_name}' を削除しました")
    except ValueError as e:
        print(f"既存のコレクション '{collection_name}' は存在しませんでした")
    except Exception as e:
        print(f"コレクションの削除中にエラーが発生しました: {str(e)}")

    vector_store = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "ip", "hnsw:search_ef": 100} 
    )
    print(f"コレクション '{collection_name}' を作成しました")

    collections = persistent_client.list_collections()
    print("現在のコレクション一覧:")
    for collection in collections:
        print(f"- {collection.name}")

    # ドキュメントをバッチで追加
    print(f"コレクション '{collection_name}' にデータをロードします")
    total_added = 0
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vector_store.add_documents(documents=batch, embedding=embeddings)
        total_added += len(batch)
        print(f"バッチ {i//batch_size + 1} を追加しました（{len(batch)}件）")
    
    print(f"\n合計 {total_added} 件のドキュメントを追加しました")

except Exception as e:
    print(f"Chromaデータベースの操作中にエラーが発生しました: {str(e)}")


既存のコレクション 'mycollection_200_ef_100' は存在しませんでした
コレクション 'mycollection_200_ef_100' を作成しました
現在のコレクション一覧:
- mycollection_200_ef_100
- mycollection_1000
- mycollection_200
- mycollection_500
コレクション 'mycollection_200_ef_100' にデータをロードします
バッチ 1 を追加しました（500件）
バッチ 2 を追加しました（500件）
バッチ 3 を追加しました（500件）
バッチ 4 を追加しました（500件）
バッチ 5 を追加しました（500件）
バッチ 6 を追加しました（500件）
バッチ 7 を追加しました（378件）

合計 3378 件のドキュメントを追加しました


### ベクトル検索のテスト

In [12]:
result = vector_store.similarity_search(query, k=top_k)

In [13]:
for i, doc in enumerate(result[:top_k]):
    print(f"\n=== 検索結果 {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")


=== 検索結果 1 ===
Content: 新たなリーダーとなったひめなは静⾹に⼿を組むことを持ちかけるが拒否される。ひめなの指⽰のもと、⽻根の神楽燦と遊狩ミユリがちはるの腕を切り落とし、⽯を強奪する。那由他は伯⽗である灯花の⽗に⼿はずを整えてもらい、同じく⽗の⾏⽅を探っている離婚した⺟と再会。⽗に関する調査資料を⼊⼿し、「湯国市」に⼿がかりがあるとわかって⾏動しようとする。
文字数: 167
Source: data\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf

=== 検索結果 2 ===
Content: 東京都在住。44歳。表の職業は国防軍を得意先とする⼟⽊建設会社のオーナーである。七草家と共に伊⾖を含む関東地⽅を監視‧守護している。⼗⽂字家は有事の実戦闘要員としての性質が強い。三年前から⼗⽂字家固有の魔法⼒低下の病に罹っており、2097年2⽉の師族会議の三ヶ⽉前に魔法技能を失っている。会議の冒頭で克⼈に当主の座を継承させることを申し出て、満場⼀致で承認される。
文字数: 183
Source: data\Wikipedia-魔法科高校の劣等生.pdf

=== 検索結果 3 ===
Content: 彼がこの世界に来た理由。それは彼が発掘したロストロギア（異世界に存在した⾼度な魔法技術の遺産）「ジュエルシード」が散らばってしまったためであった。成り⾏きから事情を知ったなのはは、ユーノと共にジュエルシードを集め、封印する⼿伝いをすることになる。そして次第になのははユーノだけではなく、⾃分のために魔法の世界に関わっていく決意を固め、その秘めた⼒を開花させていく。
文字数: 182
Source: data\Wikipedia-魔法少女リリカルなのは.pdf

=== 検索結果 4 ===
Content: 敵対していたフェイトやヴィータなどの⼈物とも、幾度となく互いの信念をかけた戦いを繰り返した結果、最終的には分かり合うことが出来ており、彼⼥達と良き友となっている。使⽤デバイス（魔法の杖）はインテリジェントデバイス「レイジングハート」。ユーノから譲り受けたものだが、彼がこのデバイスを⼿に⼊れた経緯は不明。
文字数: 152
Source: data\Wikipedia-高町なのは.pdf

=== 検索結果 5 ===
Co

: 